In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Use BigQuery DataFrames to visualize COVID-19 data

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/tree/main/notebooks/dataframes/bq_dataframes_covid_line_graphs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/dataframes/bq_dataframes_covid_line_graphs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>                                                                                          
</table>

## Overview

The goal of this notebook is to demonstrate creating line graphs from a ~20 million-row BigQuery dataset using BigQuery DataFrames. We will first create a plain line graph using matplotlip, then we will downsample and download our data to create a graph with a line of best fit using seaborn.

If you're like me, during 2020 (and/or later years) you often found yourself looking at charts like [these](https://health.google.com/covid-19/open-data/explorer/statistics) visualizing COVID-19 cases over time. For our first graph, we're going to recreate one of those charts by filtering,  summing, and then graphing COVID-19 data from the United States. BigQuery DataFrame's default integration with matplotlib will get us a satisfying result for this first graph.

For our second graph, though, we want to use a scatterplot with a line of best fit, something that matplotlib will not do for us automatically. So, we'll demonstrate how to downsample our data and use seaborn to make our plot. Our second graph will be of symptom-related search trends against new cases of COVID-19, so we'll see if searches for things like "cough" and "fever" are more common in the places and times where more new cases of COVID-19 occur.

### Dataset

This notebook uses the [BigQuery COVID-19 Open Data](https://pantheon.corp.google.com/marketplace/product/bigquery-public-datasets/covid19-open-data).

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery (compute)

Learn about [BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the BigQuery API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}

#### Set the region

You can also change the `REGION` variable used by BigQuery. Learn more about [BigQuery regions](https://cloud.google.com/bigquery/docs/locations#supported_locations).

In [ ]:
REGION = "US"  # @param {type: "string"}

Now we are ready to use BigQuery DataFrames!

## Visualization #1: Cases over time in the US

### Setup, filter data

First, let's do project setup. We use options to tell BigQuery DataFrames what project and what region to use for our cloud computing.

In [ ]:
import bigframes.pandas as bf

bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

Next, we read the data from a publicly available BigQuery dataset. This will take ~1 minute.

In [ ]:
all_data = bf.read_gbq("bigquery-public-data.covid19_open_data.covid19_open_data")

Using pandas syntax, we will select from our all_data input dataframe only those rows where the country_code is US. This is called row filtering.

In [ ]:
usa_data = all_data[all_data["country_code"] == "US"]

We're only concerned with the date and the total number of confirmed cases for now, so select just those two columns as well.

In [ ]:
usa_data = usa_data[["date", "new_confirmed"]]

### Sum data

`usa_data.groupby("date")` will give us a groupby object that lets us perform operations on groups of rows with the same date. We call sum on that object to get the sum for each day. This process might be familiar to pandas users.

In [ ]:
# numeric_only = True because we don't want to sum dates
new_cases_usa = usa_data.groupby("date").sum(numeric_only = True)

### Line graph

BigQuery DataFrames implements some of the interface required by matplotlib. This means we can pass our DataFrame right into `pyplot.plt` and using the default settings, matplotlib will draw a simple line graph for us.

In [ ]:
import matplotlib.pyplot as plt

# matplotlin will draw a line graph by default
plt.plot(new_cases_usa)
# Rotate the labels on the x axis so that they don't overlap
plt.xticks(rotation=45)
# label the y axis for clarity
plt.ylabel("New Cases")

# Show the plot
plt.show()

## Visualization #2: Symptom-related searches compared to new cases

### Filter data

We're curious if searches for symptoms like "cough" and "fever" went up in the same times and places that new COVID-19 cases occured, compared to non-symptoms like "bruise." Let's plot searches vs. new cases to see if it looks like there's a correlation.

First, we select the new cases column and the search trends we're interested in.

In [ ]:
symptom_data = all_data[["new_confirmed", "search_trends_cough", "search_trends_fever", "search_trends_bruise"]]

Not all rows have data for all of these columns, so let's select only the rows that do.

In [ ]:
symptom_data = symptom_data[
    symptom_data["search_trends_cough"].notna() &
    symptom_data["search_trends_fever"].notna() &
    symptom_data["search_trends_bruise"].notna() &
    symptom_data["new_confirmed"].notna()
]

We want to use a line of best fit to make the correlation stand out. Matplotlib does not include a feature for lines of best fit, but seaborn, which is built on matplotlib, does.

BigQuery DataFrames does not currently integrate with seaborn by default. So we will demonstrate how to downsample and download a DataFrame, and use seaborn on the downloaded data.

### Downsample and download

BigQuery DataFrames options let us set up the sampling functionality we need. Calls to `to_pandas()` usually download all the data available in our BigQuery table and store it locally as a pandas DataFrame. `pd.options.sampling.enable_downsampling = True` will make future calls to `to_pandas` use downsampling to download only part of the data, and `pd.options.sampling.max_download_size` allows us to set the amount of data to download.

In [ ]:
bf.options.sampling.enable_downsampling = True # enable downsampling
bf.options.sampling.max_download_size = 5 # download only 5 mb of data

Download the data and note the message letting us know that downsampling is being used.

In [ ]:
local_symptom_data = symptom_data.to_pandas(sampling_method="uniform")

### Graph with lines of best fit

We will now use seaborn to make the plots with the lines of best fit for cough, fever, and bruise. Note that since we're working with a local pandas dataframe, you could use any other Python library or technique you're familiar with, but we'll stick to seaborn for this notebook.

Seaborn will take a few seconds to calculate the lines. Since cough and fever are symptoms of COVID-19, but bruising isn't, we expect the slope of the line of best fit to be positive in the first two graphs, but not the third, indicating that there is a correlation between new COVID-19 cases and cough- and fever-related searches.

In [ ]:
import seaborn as sns

# first, convert to a data type that is suitable for seaborn
local_symptom_data["new_confirmed"] = \
  local_symptom_data["new_confirmed"].astype(float)
local_symptom_data["search_trends_cough"] = \
  local_symptom_data["search_trends_cough"].astype(float)

# draw the graph. This might take ~30 seconds.
sns.regplot(x="new_confirmed", y="search_trends_cough", data=local_symptom_data)

In [ ]:
# similarly, for fever

local_symptom_data["search_trends_fever"] = \
  local_symptom_data["search_trends_fever"].astype(float)
sns.regplot(x="new_confirmed", y="search_trends_fever", data=local_symptom_data)

In [ ]:
# similarly, for bruise
local_symptom_data["search_trends_bruise"] = \
  local_symptom_data["search_trends_bruise"].astype(float)
sns.regplot(
    x="new_confirmed",
    y="search_trends_bruise",
    data=local_symptom_data
)

We see that the slope of the line is positive in the graphs for cough and fever, but flat for bruise. That means that in places with increasing new cases of COVID-19, we saw increasing searches for cough and fever, but we didn't see increasing searches for unrelated symptoms like bruises. Interesting!

## Recap

We used matplotlib to draw a line graph of COVID-19 cases over time in the USA. Then, we used downsampling to download only a portion of the available data, and used seaborn locally to plot lines of best fit to observe corellation between COVID-19 cases and searches for related vs. unrelated symptoms.

Thank you for using BigQuery DataFrames!